In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'
import time

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
conf = SparkConf() \
    .setAppName("Streaming test") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1")
sc = SparkContext(conf=conf) 
sqlContext=SQLContext(sc)
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from datetime import timedelta 

In [3]:
def saveToCassandra(rows):
    if not rows.isEmpty(): 
        sqlContext.createDataFrame(rows).write\
        .format("org.apache.spark.sql.cassandra")\
        .mode('append')\
        .options(table="sent_received", keyspace="test_time")\
        .save()

In [4]:
ssc = StreamingContext(sc, 5)
kvs = KafkaUtils.createStream(ssc, "127.0.0.1:2181", "spark-streaming-consumer", {'test': 1})
data = kvs.map(lambda x: x[1])
rows= data.map(lambda x:Row(time_sent=x,time_received=time.strptime(x,'%Y-%m-%d %H:%M:%S')>time.localtime(time.time()-5)))
rows.foreachRDD(saveToCassandra)

In [5]:
ssc.start()

In [6]:
data=sqlContext.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="sent_received", keyspace="test_time")\
    .load()

In [16]:
data.registerTempTable("sent_received");
data.printSchema()
data=sqlContext.sql("select * from sent_received order by time_sent desc")
data.show()

root
 |-- time_sent: string (nullable = true)
 |-- time_received: string (nullable = true)

+-------------------+-------------+
|          time_sent|time_received|
+-------------------+-------------+
|2018-10-16 18:07:34|         true|
|2018-10-16 18:07:33|         true|
|2018-10-16 18:07:32|         true|
|2018-10-16 18:07:31|         true|
|2018-10-16 18:07:30|        false|
|2018-10-16 18:07:29|         true|
|2018-10-16 18:07:28|         true|
|2018-10-16 18:07:27|         true|
|2018-10-16 18:07:26|         true|
|2018-10-16 18:07:25|        false|
|2018-10-16 18:07:24|         true|
|2018-10-16 18:07:23|         true|
|2018-10-16 18:07:22|         true|
|2018-10-16 18:07:21|         true|
|2018-10-16 18:07:20|        false|
|2018-10-16 18:07:19|         true|
|2018-10-16 18:07:18|         true|
|2018-10-16 18:07:17|         true|
|2018-10-16 18:07:16|        false|
|2018-10-16 18:07:15|        false|
+-------------------+-------------+
only showing top 20 rows

